In [1]:
# !pip install -U chardet urllib3 gensim

In [2]:
cd ../src

/home/datvodinh/Workspace/otto-recsys/kaggle_otto_rs/src


/home/datvodinh/miniconda3/envs/kaggle-otto/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import gc
import sys
import cudf
import json
import glob
import numba
import pickle
import warnings
import itertools
import numpy as np
import pandas as pd

from tqdm import tqdm
from datetime import datetime
from collections import Counter
from numerize.numerize import numerize

from gensim.models import Word2Vec
from gensim.test.utils import common_texts

warnings.simplefilter(action="ignore", category=FutureWarning)

In [5]:
from params import *

### Params

In [6]:
MODE = "test"

In [7]:
if MODE == "val":
    files = glob.glob("../output/full_train_parquet/*") + glob.glob(
        "../output/val_parquet/*"
    )
elif MODE == "test":
    files = glob.glob("../output/full_train_val_parquet/*") + glob.glob(
        "../output/test_parquet/*"
    )
else:
    raise NotImplementedError

In [8]:
sessions = cudf.concat(
    [cudf.read_parquet(f) for f in files], ignore_index=True
)
sessions.head()

,session,aid,ts,type
0,900000,1632747,1659374717945,clicks
1,900000,1639219,1659374740973,clicks
2,900001,1466881,1659374717981,clicks
3,900001,76224,1659374841809,clicks
4,900001,1466881,1659374900554,clicks


In [9]:
sessions = sessions.sort_values(['session', 'ts'])
sessions = sessions[['session', 'aid']].groupby(['session']).agg(list)

sentences = sessions['aid'].to_pandas().to_list()

del sessions
gc.collect()
numba.cuda.current_context().deallocations.clear()

In [10]:
[len(s) for s in sentences[:5]]

[276, 32, 33, 226, 19]

In [11]:
%%time
sessions = sessions.sort_values(["session", "ts"])
sessions = sessions[["session", "aid"]].groupby(["session"]).agg(list)

sentences = sessions["aid"].to_pandas().to_list()

w2vec = Word2Vec(
    sentences=sentences, 
    vector_size=50, 
    epochs=5, 
    sg=1, 
    window=3, 
    sample=1e-3, 
    ns_exponent=1, 
    min_count=1, 
    workers=8
)

CPU times: user 1h 48min 14s, sys: 4.81 s, total: 1h 48min 19s
Wall time: 14min 27s


In [12]:
aid2idx = {aid: i for i, aid in enumerate(w2vec.wv.index_to_key)}

In [13]:
emb = {}
for aid, idx in aid2idx.items():
    emb[aid] = w2vec.wv.vectors[idx]

In [14]:
pickle.dump(
    emb,
    open(f'../output/matrix_factorization/word2vec_{MODE}.emb', 'wb')
)

np.save(f'../output/matrix_factorization/word2vec_{MODE}.npy', w2vec.wv.vectors)

Done !